In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt

# 1. 处理成分数据

In [2]:
# def parse_elements(composition):
#     pattern = r"\(([A-Za-z\d.]+)\)([\d.]+)|([A-Z][a-z]*)([\d.]+)?"
#     matches = re.findall(pattern, composition)
#     elements = {}
#     for match in matches:
#         if match[0]:  # 如果有括号
#             sub_scale_factor = sum(float(x[1]) if x[1] else 1 for x in re.findall(r"([A-Z][a-z]*)([\d.]+)?", match[0]))
#             scale_factor = float(match[1]) / sub_scale_factor
#             sub_matches = re.findall(r"([A-Z][a-z]*)([\d.]+)?", match[0])
#             for sub_match in sub_matches:
#                 if sub_match[0] not in elements:
#                     elements[sub_match[0]] = float(sub_match[1]) * scale_factor if sub_match[1] else scale_factor
#         elif match[2]:  # 如果没有括号
#             if match[2] not in elements:
#                 elements[match[2]] = float(match[3]) if match[3] else 1
#     sorted_elements = {key: value for key, value in sorted(elements.items(), key=lambda item: item[1], reverse=True)}
#     element_strings = [f"{element}{round(value,2)}" for element, value in sorted_elements.items()]
#     chem = "".join(element_strings)
#     return elements, chem

def parse_elements(composition):
    pattern = r"\[([A-Za-z\d.\(\)]+)]([\d.]+)|\(([A-Za-z\d.]+)\)([\d.]+)|([A-Z][a-z]*)([\d.]+)?"
    matches = re.findall(pattern, composition)
    elements = {}

    for match in matches:
        if match[0]:  # 如果有方括号
            bracket_scale_factor = float(match[1]) /100
            bracket_content = match[0]
            bracket_matches = re.findall(pattern, bracket_content)

            for bracket_match in bracket_matches:
                if bracket_match[2]:  # 如果有括号
                    paren_scale_factor = float(bracket_match[3]) /100
                    paren_content = bracket_match[2]
                    paren_matches = re.findall(r"([A-Z][a-z]*)([\d.]+)?", paren_content)

                    for element, weight in paren_matches:
                        scaled_weight = float(weight) /100 * paren_scale_factor * bracket_scale_factor if weight else paren_scale_factor * bracket_scale_factor

                        if element not in elements:
                            elements[element] = scaled_weight
                        else:
                            elements[element] += scaled_weight
                        
                elif bracket_match[4]:  # 如果没有括号
                    element, weight = bracket_match[4], bracket_match[5]
                    scaled_weight = float(weight) /100 * bracket_scale_factor if weight else bracket_scale_factor

                    if element not in elements:
                        elements[element] = scaled_weight
                    else:
                        elements[element] += scaled_weight

        elif match[2]:  # 如果有括号，但没有方括号
            paren_scale_factor = float(match[3]) / 100
            paren_content = match[2]
            paren_matches = re.findall(r"([A-Z][a-z]*)([\d.]+)?", paren_content)

            for element, weight in paren_matches:
                scaled_weight = float(weight) * paren_scale_factor if weight else paren_scale_factor

                if element not in elements:
                    elements[element] = scaled_weight
                else:
                    elements[element] += scaled_weight

        elif match[4]:  # 如果没有括号和方括号
            element, weight = match[4], match[5]
            scaled_weight = float(weight) /100 if weight else 1

            if element not in elements:
                elements[element] = scaled_weight
            else:
                elements[element] += scaled_weight

    sorted_elements = {key: value for key, value in sorted(elements.items(), key=lambda item: item[1], reverse=True)}
    element_strings = [f"{element}{round(value,2)}" for element, value in sorted_elements.items()]
    chem = "".join(element_strings)
    return elements, chem


def normalize_element_string(element_string):
    # 用正则表达式找出元素和它们的数字部分，例如 [('Al', '0'), ('Fe', ''), ('Mn', ''), ('Ni', ''), ('Cr', ''), ('Cu', '0.5')]
    element_parts = re.findall(r'([A-Z][a-z]*)(\d*\.?\d*)', element_string)

    element_dict = {}
    total = 0

    for element, number in element_parts:
        # 将空数字字符串（表示1）转换为实际数字
        value = float(number) if number else 1
        element_dict[element] = value
        total += value

    # 归一化占比并乘以100
    for element, value in element_dict.items():
        element_dict[element] = (value / total) * 100
        
    sorted_elements = {key: value for key, value in sorted(element_dict.items(), key=lambda item: item[1], reverse=True)}
    element_strings = [f"{element}{round(value,2)}" for element, value in sorted_elements.items()]
    chem = "".join(element_strings)
    return element_dict, chem


# 使用parse_elements和check_sum函数解析和检查成分
def parse_and_check(composition):
    elements, chem = parse_elements(composition)
    sumvalue = round(sum(elements.values()), 2)
    if abs(sumvalue  - 100) > 0.5:
        print(f"Warning: {composition} is not normalized. Sum is {sumvalue}")
    return elements, chem, abs(sumvalue - 100) < 0.5

def normalize_and_check(composition):
    elements, chem = normalize_element_string(composition)
    sumvalue = round(sum(elements.values()), 2)
    if abs(sumvalue  - 100) > 0.5:
        print(f"Warning: {composition} is not normalized. Sum is {sumvalue}")
    return elements, chem, abs(sumvalue - 100) < 0.5

In [3]:
parse_and_check("(CoCrFeMn)90Ni10")
# normalize_and_check("Al0FeMnNiCrCu0.5")

({'Co': 0.9, 'Cr': 0.9, 'Fe': 0.9, 'Mn': 0.9, 'Ni': 0.1},
 'Co0.9Cr0.9Fe0.9Mn0.9Ni0.1',
 False)

In [10]:
file_path = "D:/yuyouyu/SHU/Transfer_Learning/HEA/Data/Xiong's verification data/UTS.xlsx"
data_o = pd.read_excel(file_path)
data_o_copy = data_o.copy()
record_chem = {}
for index, row in data_o.iterrows():
    composition = row["formula"]
    if composition in record_chem:
        pass
        # print(record_chem[composition], index, composition)
    else:
        record_chem[composition] = index
    elements, chem, is_sum_100 = normalize_and_check(composition)

    if not is_sum_100:
        print(f"index {index} sum {is_sum_100}:\n{row}\n")
    data_o.loc[index, "Chemical compostion"] = chem
    # 将新的成分列添加到DataFrame中
    for element, value in elements.items():
        if element not in data_o.columns:
            data_o[element] = 0
        data_o.loc[index, element] = value

In [11]:
# data_o.replace("–", "", inplace=True)
# data_o.drop_duplicates(subset='Chemical compostion', keep='first', inplace=True)
data_o.to_excel(file_path, index=False)

In [10]:
data_o = pd.read_excel(file_path)
data_o.describe()
# data_o.describe().to_excel(f"{file_path}_des.xlsx")

,EL,cr,TAN,tAN,Co,Fe,Ni,Al,Cr,Mo
count,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,17.200000,36.00000,295.000000,0.400000,10.639010,20.792856,36.087603,14.230846,13.634300,4.615385
std,13.608821,49.29503,376.414001,0.547723,9.849371,5.895217,4.227088,8.097290,7.821535,10.320314
min,6.000000,0.00000,25.000000,0.000000,0.000000,16.393443,30.769231,0.000000,0.000000,0.000000
25%,8.000000,0.00000,25.000000,0.000000,0.000000,16.393443,34.426230,16.393443,15.384615,0.000000
50%,14.000000,0.00000,25.000000,0.000000,16.393443,20.000000,34.426230,16.393443,16.393443,0.000000
75%,18.000000,90.00000,600.000000,1.000000,16.393443,20.408163,40.000000,18.367347,16.393443,0.000000
max,40.000000,90.00000,800.000000,1.000000,20.408163,30.769231,40.816327,20.000000,20.000000,23.076923


# 2. 绘制数据分析图

In [ ]:
# 设置箱线图样式
boxprops = dict(linestyle='-', linewidth=2, color='black')
whiskerprops = dict(linestyle='--', linewidth=1.5, color='gray')
flierprops = dict(marker='o', markersize=5, markerfacecolor='red', alpha=0.7)
medianprops = dict(linestyle='-', linewidth=2, color='blue')

def draw_box(data, target_name):
    plt.cla()
    data_d = [data]

    # 设置每组数据对应的标签
    labels = [target_name]

    # 绘制箱线图
    plt.boxplot(data_d, vert=False, showmeans=True, meanline=True,
                boxprops=boxprops, whiskerprops=whiskerprops, flierprops=flierprops, medianprops=medianprops, labels=labels)

    # 添加标题和标签
    plt.title(f'{target_name} Target Distributions(Filter)')
    plt.xlabel('Range')

    # 显示图形
    plt.savefig(f"./Wang's Data/{target_name}_box.png", dpi=300)

In [ ]:
data = pd.read_excel("./Wang's Data/data_processed.xlsx")
target_names = ['E(GPa)', 'Tg(K)', 'Tm(K)', 'ry(GPa)', 'Tx(K)', 'Tl(K)', 'dc(mm)']
for target in target_names:
    non_empty_values = data.dropna(subset=[target])[target]
    draw_box(non_empty_values, target.replace('/', '_'))

In [ ]:
data_f = data.drop(target_names, axis=1, inplace=False)
data_f = data_f.drop(['Chemical compostion（at.%）', "Chemical compostion"], axis=1, inplace=False)
element_count = (data_f != 0).sum()
fig, ax = plt.subplots(figsize=(20, 12))
ax.bar(element_count.index, element_count.values)
ax.set_xlabel("Elements")
ax.set_ylabel("Frequency")
ax.set_title("Element Frequency in Compositions")
plt.savefig(f"./Wang's Data/element_statis.png", dpi=300)

In [ ]:
properties = ['E(GPa)', 'Tg(K)', 'Tm(K)', 'ry(GPa)', 'Tx(K)', 'Tl(K)', 'dc(mm)']

for prop in properties:
    fig, ax = plt.subplots()
    non_empty_values = data.dropna(subset=[prop])[prop]
    ax.hist(non_empty_values, bins=10, alpha=0.7)
    ax.set_title(f"{prop} Distribution")
    ax.set_xlabel(prop)
    ax.set_ylabel("Frequency")
    prop = prop.replace('/', '_')
    # 保存每个子图为单独的图片
    plt.savefig(f"./Wang's Data/{prop}_distribution.png")
    plt.close(fig)